In [1]:
#Start storing all games from each day, fill in win or loss and run prediction from that
#format data from odd shark to include same info as bets.xlsx

import requests 
import re
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import re
pd.set_option('display.max_rows', 500)
from datetime import date
from sklearn.ensemble import RandomForestClassifier

In [2]:
url='https://www.oddsshark.com/ncaab/computer-picks'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')


In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:" Jan 28 @ 6:30 PM"
2:"ATS"
3:"Total"


In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
[len(C) for (title,C) in col]


[109, 109, 109]

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
df


,Jan 28 @ 6:30 PM,ATS,Total
0,Predicted Score,75.1 - 66,141.1
1,Computer Pick,VILLA (-3.5),Under 147.5
2,Public Consensus,VILLA (-3.5),Under 147.5
3,Consensus Bet %,65%,67%
4,Jan 28 @ 6:30 PM,ATS,Total
5,Predicted Score,63.5 - 69.6,133.1
6,Computer Pick,TEXAM (+10),Over 120
7,Public Consensus,TENN (-10),Over 120
8,Consensus Bet %,51%,64%
9,Jan 28 @ 7:00 PM,ATS,Total


In [9]:
comp = df.iloc[1:,:]
comp_ = comp.iloc[::5,1]
comp_ = comp_.reset_index()
comp_ = comp_.drop(columns=['index'])

In [10]:
td_elements = doc.xpath('//caption')
[len(T) for T in td_elements[:12]]
#Create empty list
new_col=[]
z=0
#For each row, store each first element (header) and an empty list
for t in td_elements[0]:
    z+=1
    name=t.text_content()
    print('%d:"%s"'%(z,name))
    new_col.append((name,[]))
    
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(td_elements)):
    #T is our j'th row
    T=td_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=1:
        break
    
    #i is the index of our column
    z=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if z>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        new_col[z][1].append(data)
        #Increment i for the next column
        z+=1

1:" VILLA Villanova vs STJON St. John's Matchup"


In [11]:
Dict1={title:column for (title,column) in new_col}
df1=pd.DataFrame(Dict1)
df2 = pd.melt(df1)
row1 = df2['variable']
games = df2[['value']]
games_ = games.shift(1, axis=0)
games_.iloc[0] = df1.columns

In [12]:
scores = df.iloc[::5,:]
scores = scores.reset_index()
scores_ = scores.drop(columns=['index'])

In [13]:
mergedDf = games_.merge(scores_, left_index=True, right_index=True)

In [14]:
mergedDf1 = mergedDf.merge(comp_, left_index=True, right_index=True)


In [15]:
#misst to win and cover
#flst to win
#mich to win
#syr to win
#cmich to win
#ball to win
#toldo to win
#nill to win
#rutgr to win
#miami to win and cover
#ga to wn and cover
#butlr to win
#frsno to win and cover

In [17]:
new = mergedDf1["ATS_y"].str.split(" ", n = -1, expand = True) 
spread = new[2]
mergedDf1['spread'] = spread
mergedDf1

,value,Jan 28 @ 6:30 PM,ATS_x,Total,ATS_y,spread
0,VILLA Villanova vs STJON St. John's Matchup,Predicted Score,75.1 - 66,141.1,VILLA (-3.5),(-3.5)
1,TEXAM Texas A&M vs TENN Tennessee Matchup,Predicted Score,63.5 - 69.6,133.1,TEXAM (+10),(+10)
2,MISST Mississippi State vs FLA Florida Matchup,Predicted Score,71.4 - 71,142.4,MISST (+6.5),(+6.5)
3,SMU Southern Methodist vs CINCY Cincinnati Ma...,Predicted Score,66.4 - 74.4,140.8,CINCY (-5),(-5)
4,FLAST Florida State vs VIRGA Virginia Matchup,Predicted Score,68.9 - 63,131.9,FLAST (-1.5),(-1.5)
5,MICH Michigan vs NEBRS Nebraska Matchup,Predicted Score,75.1 - 73.5,148.6,NEBRS (+4.5),(+4.5)
6,SYRA Syracuse vs CLEMS Clemson Matchup,Predicted Score,75.2 - 66.2,141.4,SYRA (+2),(+2)
7,RHODEI Rhode Island vs GMASN George Mason Mat...,Predicted Score,73 - 62.4,135.4,RHODEI (-4.5),(-4.5)
8,RICHM Richmond vs VACOM VCU Matchup,Predicted Score,68.2 - 75.2,143.4,RICHM (+9),(+9)
9,CMICH Central Michigan vs MIAOH Miami-Ohio Ma...,Predicted Score,73.9 - 71.2,145.1,CMICH (+1),(+1)


In [649]:
mergedDf1.to_excel('ncaab-{}.xlsx'.format(date.today()))

In [936]:
outcome = pd.read_excel('bets.xlsx')
outcome.groupby(['Sport','win_or_loss'])['outcome'].sum()

Sport  win_or_loss
nba    l              -4.500000
       w              10.117775
ncaab  l             -30.500000
       w              36.090618
nhl    l              -1.500000
       w               0.611111
Name: outcome, dtype: float64

In [941]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
X = outcome[['sportid','betid','h_o_a_id','odds','predicted win amount','w_o_a_fav','spread','TotalBuffer']]
y = outcome[['Outcomeid']]
X1 = X.iloc[:160,:]
y1 = y.iloc[:160,:]
X_train,X_test, y_train, y_test = train_test_split(X1,y1,random_state=0)
forest = RandomForestClassifier(random_state = 1, n_estimators = 10, min_samples_split = 2)
forest.fit(X_train, y_train)

<ipython-input-941-d11187ec10e3>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [948]:
print('Accuracy on training set: {:.2f}'.format(forest.score(X_train,y_train)))
print('Accuracy on test set: {:.2f}'.format(forest.score(X_test, y_test)))
pred = X.iloc[160:, :]
pred

Accuracy on training set: 0.98
Accuracy on test set: 0.62


,sportid,betid,h_o_a_id,odds,predicted win amount,w_o_a_fav,spread,TotalBuffer
160,1,1,0,125,13.2,1,2.0,15.2
161,1,2,0,-115,0.4,2,7.0,7.4
162,1,1,0,255,0.4,1,7.0,7.4
163,1,1,0,105,0.6,1,2.0,2.6
164,1,1,1,-105,0.2,0,1.0,1.2
165,2,1,1,105,3.6,1,1.0,4.6
166,2,1,1,-125,5.8,0,1.5,7.3


In [949]:
print(pred)
print(gbrt.predict_proba(pred))

     sportid  betid  h_o_a_id  odds  predicted win amount  w_o_a_fav  spread  \
160        1      1         0   125                  13.2          1     2.0   
161        1      2         0  -115                   0.4          2     7.0   
162        1      1         0   255                   0.4          1     7.0   
163        1      1         0   105                   0.6          1     2.0   
164        1      1         1  -105                   0.2          0     1.0   
165        2      1         1   105                   3.6          1     1.0   
166        2      1         1  -125                   5.8          0     1.5   

     TotalBuffer  
160         15.2  
161          7.4  
162          7.4  
163          2.6  
164          1.2  
165          4.6  
166          7.3  
[[0.52849655 0.47150345]
 [0.54006258 0.45993742]
 [0.53209849 0.46790151]
 [0.54006258 0.45993742]
 [0.54006258 0.45993742]
 [0.47947726 0.52052274]
 [0.47947726 0.52052274]]


0.08147554984966127